$
Model \ for \ Tensorflow
$

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 as cv
import time
import timeit

print(tf.__version__)

2.7.0


Load data

In [2]:
datasetfilename = 'master_dataset.npz'

with np.load(datasetfilename, allow_pickle=True) as data:
    dataImages = data['images']
    dataLabel = data['labels']
    dataLabelNames = data['labelnames']

classNames = sorted(np.unique(dataLabelNames))
classLabels = sorted(np.unique(dataLabel))

print(classNames)
print(classLabels)

['afiq', 'azureen', 'gavin', 'goke', 'inamul', 'jincheng', 'mahmuda', 'numan', 'saseendran']
[0, 1, 2, 3, 4, 5, 6, 7, 8]


### Convert size

In [4]:
N = len(dataImages)
shape = (N, 200,200,3)

y = np.empty(shape, dtype='uint8')

for i in range(N):
    y[i] = cv.resize(dataImages[i], [200,200], interpolation=cv.INTER_NEAREST)
    
dataImages = y

dataLabel = dataLabel.astype('uint8')

trainImages, testImages, trainLabels, testLabels = train_test_split(dataImages, dataLabel, test_size=0.3, random_state=42)

for i in range(len(trainImages)):
    trainImages[i] = cv.resize(trainImages[i], [200,200], interpolation=cv.INTER_NEAREST)


In [5]:
###
inputShape = trainImages[0].shape
outputShape = len(classNames)

trainImages = trainImages / 255
testImages = testImages / 255

### Create model

In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=inputShape),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(outputShape)
])

In [7]:
model.summary()

model.compile(optimizer='adam', loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(trainImages, trainLabels, epochs=30)

testLoss, testAcc = model.evaluate(testImages, testLabels, verbose=2)

print('\nTest accuracy:', testAcc)

predictions = model.predict(testImages)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 120000)            0         
                                                                 
 dense (Dense)               (None, 512)               61440512  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 9)                 4617      
                                                                 
Total params: 61,707,785
Trainable params: 61,707,785
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
10/10 [==============================] - 86s 3s/step - loss: 85.2787 - accuracy: 0.1926
Epoch 2/30
10/10 [==============================] - 9s 813ms/step - loss: 13.

In [8]:
import os 

maxIterations = 5
testAccList = []
thresholdAcc = 0.90
lastTestAcc = 0.0

model = None
testLoss = 0.0
testAcc = 0.0
modelDir = 'tf_model'

start = time.time() 

In [9]:
for iter in range(maxIterations):
    
    print(f'Simulation {iter+1}/{maxIterations}', end='... ')
    
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=inputShape),
        tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(outputShape)
    ])

    # model.summary()

    model.compile(optimizer='adam',
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])

    model.fit(trainImages, trainLabels, epochs = 10, verbose=0)

    testLoss, testAcc = model.evaluate(testImages, testLabels, verbose=0)

    testAccList.append(testAcc)

    # print('\ntest accuracy:',testAcc)
    print(f'test accuracy {testAcc}', end='... ')
    

    exportPath = ''

    # Save model if greater than threshold-accuracy 0.9
    if testAcc > thresholdAcc:
        # SavedModel format
        version = f'tf_{testAcc*100:.0f}'

        # for SavedModel format
        exportPath = os.path.join(modelDir, version)

        # Save the model
        model.save(exportPath, save_format = 'tf')
        print(f'\nexport path = {exportPath}')

        # HDF5 format
#         exportPath = os.path.join(modelDir, f'{version}.h5')
#         # Save the entire model to a HDF5 file.
#         # The .h5 extension indicates that the model should be saved to HDF5
#         model.save(exportPath)
# #         print('Saved: ', exportPath)
#         print(f'export path {exportPath}', end='')
        

        thresholdAcc = testAcc
        
end = time.time()
times = end - start
print('Time elapsed:', times)
print('\nDone all.')

Simulation 1/5... test accuracy 0.960629940032959... INFO:tensorflow:Assets written to: tf_model\tf_96\assets

export path = tf_model\tf_96
Simulation 2/5... test accuracy 0.8818897604942322... Simulation 3/5... test accuracy 0.960629940032959... Simulation 4/5... test accuracy 0.913385808467865... Simulation 5/5... test accuracy 0.9448819160461426... Time elapsed: 209.84184503555298

Done all.
